In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style untuk plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Environment setup complete!")
print("📈 Ready untuk Grid Trading Analysis")
print("~" * 50)

✅ Environment setup complete!
📈 Ready untuk Grid Trading Analysis
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [25]:
print(f"Fetching crypto data for Grid Trading analysis...")
print(f"🗓️ Period: 2020-2024 (5 years historical data)")
print(f"🪙 Assets: BTC, ETH, BNB, ADA, SOL") # Updated based on later image
print("-" * 50)

def fetch_crypto_data(symbol, start_date, end_date):
    """
    Fetch cryptocurrency data menggunakan yfinance - Fixed version
    """
    try:
        # Mapping symbols untuk Yahoo Finance
        yahoo_symbol = f"{symbol}-USD"
        
        # Download data
        data = yf.download(yahoo_symbol, start=start_date, end=end_date)
        
        if data.empty:
            print(f"❌ No data found for {symbol}")
            return None
        
        # Handle MultiIndex columns if exists
        if hasattr(data.columns, 'levels'):
            data.columns = data.columns.droplevel(1) # Asumsi level 1 berisi Ticker

        # Clean column names - handle both string and other types
        new_columns = []
        for col in data.columns:
             if isinstance(col, str):
                 new_col = col.lower().replace(' ', '_')
             else:
                 # Coba konversi ke string jika bukan string
                 new_col = str(col).lower().replace(' ', '_')
             new_columns.append(new_col)
        data.columns = new_columns
        
        # Add symbol column
        data['symbol'] = symbol
        
        print(f"✅ ({symbol}): {len(data)} days data loaded")
        return data

    except Exception as e:
        print(f"❌ Error fetching {symbol}: {str(e)}")
        return None

# Define date range
start_date = "2020-01-01"
end_date = "2024-12-31"

# Fetch data untuk 5 cryptos (updated list based on image)
crypto_symbols = ['BTC', 'ETH', 'BNB', 'ADA', 'SOL']
crypto_data = {}

for symbol in crypto_symbols:
    print(f"\nFetching {symbol} data...")
    data = fetch_crypto_data(symbol, start_date, end_date)
    if data is not None:
        crypto_data[symbol] = data

print(f"\n✅ Data acquisition complete!")
print(f"📊 Successfully loaded {len(crypto_data)} cryptocurrencies")
print("-" * 50)

# Quick preview
if crypto_data:
    sample_symbol = list(crypto_data.keys())[0]
    print(f"\n📄 Sample data structure ({sample_symbol}):")
    print(crypto_data[sample_symbol].head())
    print(f"\n   Columns available: {list(crypto_data[sample_symbol].columns)}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Fetching crypto data for Grid Trading analysis...
🗓️ Period: 2020-2024 (5 years historical data)
🪙 Assets: BTC, ETH, BNB, ADA, SOL
--------------------------------------------------

Fetching BTC data...
✅ (BTC): 1826 days data loaded

Fetching ETH data...
✅ (ETH): 1826 days data loaded

Fetching BNB data...
✅ (BNB): 1826 days data loaded

Fetching ADA data...
✅ (ADA): 1826 days data loaded

Fetching SOL data...
✅ (SOL): 1726 days data loaded

✅ Data acquisition complete!
📊 Successfully loaded 5 cryptocurrencies
--------------------------------------------------

📄 Sample data structure (BTC):
                  close         high          low         open       volume  \
Date                                                                          
2020-01-01  7200.174316  7254.330566  7174.944336  7194.892090  18565664997   
2020-01-02  6985.470215  7212.155273  6935.270020  7202.551270  20802083465   
2020-01-03  7344.884277  7413.715332  6914.996094  6984.428711  28111481032   
2020

In [26]:
# Cell 3: Volatility Analysis untuk Grid Trading Parameters
print("📊 VOLATILITY ANALYSIS untuk Grid Trading Parameters")
print("🎯 Tujuan: Menentukan optimal grid spacing berdasarkan historical data")
print("-" * 70)

def calculate_volatility_metrics(data, symbol):
    """
    Calculate various volatility metrics untuk grid parameter optimization
    """
    # Calculate daily returns
    # Pastikan menggunakan nama kolom yang sudah dibersihkan ('close')
    if 'close' not in data.columns:
        print(f"   ⚠️ 'close' column not found for {symbol}. Skipping.")
        return None
        
    data['daily_return'] = data['close'].pct_change()

    # Remove outliers (beyond 3 standard deviations)
    std_returns = data['daily_return'].std()
    mean_returns = data['daily_return'].mean()
    # Create a cleaned series, keeping original NaNs
    data['daily_return_clean'] = data['daily_return'].where(
        abs(data['daily_return'] - mean_returns) <= 3 * std_returns
    )

    # Calculate volatility metrics using the cleaned data
    volatility_metrics = {
        'symbol': symbol,
        'daily_volatility': data['daily_return_clean'].std() * 100,
        'annual_volatility': data['daily_return_clean'].std() * np.sqrt(252) * 100, # Assume 252 trading days
        'average_daily_move': data['daily_return_clean'].abs().mean() * 100,
        'max_daily_gain': data['daily_return_clean'].max() * 100,
        'max_daily_loss': data['daily_return_clean'].min() * 100,
        'positive_days_pct': (data['daily_return_clean'] > 0).mean() * 100,
        'data_points': len(data['daily_return_clean'].dropna())
    }
    
    return volatility_metrics

# Calculate volatility untuk semua crypto
volatility_results = []
print("🔍 Calculating volatility metrics...\n")

for symbol, data in crypto_data.items():
    print(f"   Analyzing {symbol}...")
    # Make a copy to avoid modifying the original dict entry
    metrics = calculate_volatility_metrics(data.copy(), symbol) 
    if metrics:
        volatility_results.append(metrics)
        # Print summary inside the loop
        print(f"     Daily Volatility: {metrics['daily_volatility']:.2f}%")
        print(f"     Average Daily Move: {metrics['average_daily_move']:.2f}%")
        print(f"     Win Rate (Positive Days): {metrics['positive_days_pct']:.1f}%")

# Convert to DataFrame untuk analysis
volatility_df = pd.DataFrame(volatility_results)

print("\n\n" + "=" * 70)
print("📊 VOLATILITY SUMMARY TABLE:")
print("-" * 70)
if not volatility_df.empty:
    display_cols = ['symbol', 'daily_volatility', 'average_daily_move', 'positive_days_pct']
    # Filter out columns that might not exist if some symbols failed
    display_cols = [col for col in display_cols if col in volatility_df.columns] 
    print(volatility_df[display_cols].round(2))

    print(f"\n💡 GRID SPACING RECOMMENDATIONS:")
    print("-" * 50)
    print("(Rule: Grid spacing = 0.5 x Average Daily Move)")
    print("(Conservative approach untuk minimize whipsaws)")
    print()
    
    for _, row in volatility_df.iterrows():
        recommended_spacing = row['average_daily_move'] * 0.5
        print(f"{row['symbol']:<5}: {recommended_spacing:.2f}% grid spacing recommended")
else:
    print("No volatility data calculated.")

📊 VOLATILITY ANALYSIS untuk Grid Trading Parameters
🎯 Tujuan: Menentukan optimal grid spacing berdasarkan historical data
----------------------------------------------------------------------
🔍 Calculating volatility metrics...

   Analyzing BTC...
     Daily Volatility: 2.89%
     Average Daily Move: 2.09%
     Win Rate (Positive Days): 50.4%
   Analyzing ETH...
     Daily Volatility: 3.70%
     Average Daily Move: 2.73%
     Win Rate (Positive Days): 51.6%
   Analyzing BNB...
     Daily Volatility: 3.58%
     Average Daily Move: 2.58%
     Win Rate (Positive Days): 52.0%
   Analyzing ADA...
     Daily Volatility: 4.37%
     Average Daily Move: 3.24%
     Win Rate (Positive Days): 49.2%
   Analyzing SOL...
     Daily Volatility: 5.83%
     Average Daily Move: 4.37%
     Win Rate (Positive Days): 49.0%


📊 VOLATILITY SUMMARY TABLE:
----------------------------------------------------------------------
  symbol  daily_volatility  average_daily_move  positive_days_pct
0    BTC          

In [27]:
print("--- GRID TRADING CLASS IMPLEMENTATION ---")
print("🚀 Building Grid Trading Engine untuk Crypto")
print("-" * 60)

class GridTradingStrategy:
    def __init__(self, initial_capital, grid_spacing_pct, num_grids=10, fee_rate=0.001):
        """
        Initialize the Grid Trading Strategy

        Parameters:
        - initial_capital: Starting capital in USD
        - grid_spacing_pct: Grid spacing as percentage (e.g., 1.7 for 1.7%)
        - num_grids: Number of grid levels above and below current price
        - fee_rate: Trading fee as a decimal (e.g., 0.001 for 0.1%)
        """
        self.initial_capital = initial_capital
        self.current_capital = initial_capital
        self.grid_spacing_pct = grid_spacing_pct / 100 # Convert to decimal
        self.num_grids = num_grids
        
        # Trading state
        self.positions = []      # List of active buy positions
        self.grid_levels = []    # Grid price levels
        self.trades = []         # History of all trades
        self.total_fees = 0
        self.fee_rate = fee_rate # 0.1% trading fee

        print("Grid Strategy Initialized:")
        print(f"  Capital: ${initial_capital:,.2f}")
        print(f"  Grid Spacing: {grid_spacing_pct}%")
        print(f"  Grid Levels: {num_grids} above/below")

    def setup_grid_levels(self, current_price):
        """Setup grid levels around the current price"""
        self.base_price = current_price
        self.grid_levels = []

        # Allocate 80% of capital for buying, 20% kept as cash reserve
        buy_capital = self.initial_capital * 0.8
        self.capital_per_grid = buy_capital / self.num_grids

        # Create grid levels
        for i in range(-self.num_grids, self.num_grids + 1):
            if i == 0:
                continue # Skip current price level

            grid_price = current_price * (1 + i * self.grid_spacing_pct)
            self.grid_levels.append({
                'level': i,
                'price': grid_price,
                'type': 'buy' if i < 0 else 'sell',
                'executed': False
            })
            
        # Sort grids by price for easier processing
        self.grid_levels.sort(key=lambda x: x['price'])
        
        highest_sell = max(g['price'] for g in self.grid_levels if g['type'] == 'sell')
        lowest_buy = min(g['price'] for g in self.grid_levels if g['type'] == 'buy')

        print(f"\nGrid Setup Complete:")
        print(f"  Base Price: ${current_price:,.2f}")
        print(f"  Capital per Grid: ${self.capital_per_grid:,.2f}")
        print(f"  📈 Highest Sell Level: ${highest_sell:,.2f}")
        print(f"  📉 Lowest Buy Level: ${lowest_buy:,.2f}")
        
    def process_price(self, date, price):
        """Process new price and execute grid orders"""
        
        # Check for buy orders (price going down)
        for grid in self.grid_levels:
            if (grid['type'] == 'buy' and 
                not grid['executed'] and 
                price <= grid['price'] and
                self.current_capital >= self.capital_per_grid):
                
                # Execute buy order
                quantity = self.capital_per_grid / price
                fee = self.capital_per_grid * self.fee_rate

                self.positions.append({
                    'date': date,
                    'price': price,
                    'quantity': quantity,
                    'cost': self.capital_per_grid,
                    'grid_level': grid['level']
                })

                self.current_capital -= (self.capital_per_grid + fee)
                self.total_fees += fee
                grid['executed'] = True
                
                self.trades.append({
                    'date': date, 'type': 'BUY', 'price': price,
                    'quantity': quantity, 'value': self.capital_per_grid,
                    'fee': fee, 'grid_level': grid['level']
                })

        # Check for sell orders (price going up)
        positions_to_remove = []
        for i, position in enumerate(self.positions):
            # Calculate sell price for this position (buy price + one grid step up)
            sell_price = position['price'] * (1 + self.grid_spacing_pct)
            
            if price >= sell_price:
                # Execute sell order
                sell_value = position['quantity'] * price
                fee = sell_value * self.fee_rate
                profit = sell_value - position['cost'] - fee
                
                self.current_capital += (sell_value - fee)
                self.total_fees += fee
                positions_to_remove.append(i)
                
                # Find the corresponding buy grid and reset it
                for grid in self.grid_levels:
                    if grid['level'] == position['grid_level']:
                        grid['executed'] = False
                        break
                        
                self.trades.append({
                    'date': date, 'type': 'SELL', 'price': price,
                    'quantity': position['quantity'], 'value': sell_value,
                    'fee': fee, 'profit': profit
                })

        # Remove sold positions safely (in reverse order)
        for i in sorted(positions_to_remove, reverse=True):
            del self.positions[i]
            
    def get_portfolio_value(self, current_price):
        """Calculate total portfolio value"""
        cash = self.current_capital
        crypto_value = sum([pos['quantity'] * current_price for pos in self.positions])
        return cash + crypto_value

    def get_performance_summary(self, current_price):
        """Get comprehensive performance summary"""
        portfolio_value = self.get_portfolio_value(current_price)
        total_return = portfolio_value - self.initial_capital
        return_pct = (total_return / self.initial_capital) * 100

        return {
            'portfolio_value': portfolio_value,
            'total_return': total_return,
            'return_pct': return_pct,
            'cash': self.current_capital,
            'crypto_holdings': len(self.positions),
            'total_trades': len(self.trades),
            'total_fees': self.total_fees,
        }

print("\n✅ Grid Trading Class Implementation Complete!")
print("🚀 Ready untuk backtesting!")

--- GRID TRADING CLASS IMPLEMENTATION ---
🚀 Building Grid Trading Engine untuk Crypto
------------------------------------------------------------

✅ Grid Trading Class Implementation Complete!
🚀 Ready untuk backtesting!


In [28]:
print("\n--- GRID TRADING BACKTEST PREPARATION ---")
print("🗓️ Backtest Period: January - June 2025")
print("💰 Initial Capital: $1,000")
print("-" * 60)

# Prepare 2025 data untuk backtest
def get_backtest_data(symbol, start_date="2025-01-01", end_date="2025-06-30"):
    """
    Get 2025 data for backtesting
    """
    try:
        yahoo_symbol = f"{symbol}-USD"
        data = yf.download(yahoo_symbol, start=start_date, end=end_date, progress=False)
        
        if data.empty:
            print(f"❌ No 2025 data available for {symbol}")
            return None

        # Handle MultiIndex columns if exists
        if hasattr(data.columns, 'levels'):
            data.columns = data.columns.droplevel(1)

        # Clean column names
        new_columns = []
        for col in data.columns:
            if isinstance(col, str):
                new_col = col.lower().replace(' ', '_')
            else:
                new_col = str(col).lower().replace(' ', '_')
            new_columns.append(new_col)
        data.columns = new_columns
        data['symbol'] = symbol
        
        print(f"✅ {symbol}: {len(data)} days of 2025 data loaded")
        return data

    except Exception as e:
        print(f"❌ Error fetching 2025 data for {symbol}: {str(e)}")
        return None

# Fetch 2025 data untuk backtest
print("🛰️ Fetching 2025 data untuk backtest...")
backtest_data = {}

# Menggunakan `crypto_data.keys()` dari sel sebelumnya untuk mendapatkan daftar simbol
for symbol in crypto_data.keys():
    print(f"\n   Loading {symbol} 2025 data...")
    data_2025 = get_backtest_data(symbol)
    if data_2025 is not None and len(data_2025) > 0:
        backtest_data[symbol] = data_2025

# Tampilkan ringkasan data yang berhasil diunduh
print(f"\n✅ 2025 Data Summary:")
print("=" * 40)
for symbol, data in backtest_data.items():
    start_price = data['close'].iloc[0]
    end_price = data['close'].iloc[-1]
    price_change = ((end_price - start_price) / start_price) * 100
    
    print(f"{symbol}:")
    print(f"   Days: {len(data)}")
    print(f"   Start: ${start_price:,.2f}")
    print(f"   End:   ${end_price:,.2f}")
    print(f"   📈 Change: {price_change:+.2f}%")
    print()

# Grid parameters dari volatility analysis sebelumnya
grid_parameters = {
    'BTC': 1.04,
    'ETH': 1.36,
    'BNB': 1.29,
    'ADA': 1.62,
    'SOL': 2.19
}

print("⚙️ Grid Parameters untuk Backtest:")
print("=" * 40)
for symbol, spacing in grid_parameters.items():
    if symbol in backtest_data:
        print(f"{symbol}: {spacing:.2f}% grid spacing")


--- GRID TRADING BACKTEST PREPARATION ---
🗓️ Backtest Period: January - June 2025
💰 Initial Capital: $1,000
------------------------------------------------------------
🛰️ Fetching 2025 data untuk backtest...

   Loading BTC 2025 data...
✅ BTC: 180 days of 2025 data loaded

   Loading ETH 2025 data...
✅ ETH: 180 days of 2025 data loaded

   Loading BNB 2025 data...
✅ BNB: 180 days of 2025 data loaded

   Loading ADA 2025 data...
✅ ADA: 180 days of 2025 data loaded

   Loading SOL 2025 data...
✅ SOL: 180 days of 2025 data loaded

✅ 2025 Data Summary:
BTC:
   Days: 180
   Start: $94,419.76
   End:   $108,385.57
   📈 Change: +14.79%

ETH:
   Days: 180
   Start: $3,353.50
   End:   $2,500.96
   📈 Change: -25.42%

BNB:
   Days: 180
   Start: $706.51
   End:   $655.04
   📈 Change: -7.29%

ADA:
   Days: 180
   Start: $0.92
   End:   $0.58
   📈 Change: -37.00%

SOL:
   Days: 180
   Start: $193.87
   End:   $153.35
   📈 Change: -20.90%

⚙️ Grid Parameters untuk Backtest:
BTC: 1.04% grid spacin

In [29]:
print("\n--- GRID TRADING BACKTEST EXECUTION ---")
print("🎯 Testing Grid Strategy pada 5 Cryptocurrencies")
print("💰 Initial Capital: $1,000 per crypto")
print("-" * 70)

def run_grid_backtest(symbol, data, grid_spacing, initial_capital=1000):
    """
    Run comprehensive grid trading backtest
    """
    print(f"\n===== BACKTESTING {symbol} =====")
    print(f"Grid Spacing: {grid_spacing:.2f}%")
    print("-" * 40)

    # Initialize strategy
    strategy = GridTradingStrategy(
        initial_capital=initial_capital,
        grid_spacing_pct=grid_spacing,
        num_grids=8  # 8 levels above/below for better coverage
    )

    # Setup grid menggunakan first day price
    first_price = data['close'].iloc[0]
    strategy.setup_grid_levels(first_price)

    # Track performance over time
    daily_performance = []

    # Run backtest
    print(f"   Running backtest...")
    total_executed_orders = 0

    for date, row in data.iterrows():
        current_price = row['close']
        executed_orders = strategy.process_price(date, current_price)
        total_executed_orders += len(strategy.trades) - total_executed_orders # A more robust way to count
        
        # Record daily performance
        performance = strategy.get_performance_summary(current_price)
        performance['date'] = date
        performance['price'] = current_price
        daily_performance.append(performance)

    # Final performance summary
    final_performance = daily_performance[-1] if daily_performance else {}

    print(f"\n   ✅ Backtest Complete!")
    if final_performance:
        print(f"      Total Orders Executed: {len(strategy.trades)}")
        print(f"      Final Portfolio Value: ${final_performance['portfolio_value']:,.2f}")
        print(f"      Total Return: ${final_performance['total_return']:,.2f}")
        print(f"      Return Percentage: {final_performance['return_pct']:+.2f}%")
        print(f"      Cash Remaining: ${final_performance['cash']:,.2f}")
        print(f"      Crypto Holdings: {final_performance['crypto_holdings']}")
        print(f"      Total Fees Paid: ${final_performance['total_fees']:,.2f}")

    return {
        'symbol': symbol,
        'strategy': strategy,
        'daily_performance': pd.DataFrame(daily_performance),
        'final_performance': final_performance,
        'grid_spacing': grid_spacing
    }

# Run backtest untuk semua crypto
backtest_results = {}
performance_summary = []

for symbol in backtest_data.keys():
    if symbol in grid_parameters:
        result = run_grid_backtest(
            symbol=symbol,
            data=backtest_data[symbol],
            grid_spacing=grid_parameters[symbol],
            initial_capital=1000
        )
        backtest_results[symbol] = result
        
        # Add to summary
        perf = result['final_performance'].copy()
        perf['symbol'] = symbol
        perf['grid_spacing'] = grid_parameters[symbol]
        performance_summary.append(perf)

# Create summary table
summary_df = pd.DataFrame(performance_summary)

print(f"\n\n🏆 BACKTEST RESULTS SUMMARY")
print("=" * 70)
print("Grid Trading Performance (Jan-Jun 2025)")
print()

if not summary_df.empty:
    display_columns = ['symbol', 'portfolio_value', 'return_pct', 'total_trades', 'total_fees']
    formatted_summary = summary_df[display_columns].copy()
    formatted_summary['portfolio_value'] = formatted_summary['portfolio_value'].round(2)
    formatted_summary['return_pct'] = formatted_summary['return_pct'].round(2)
    formatted_summary['total_fees'] = formatted_summary['total_fees'].round(2)

    print(formatted_summary.to_string(index=False))

    # Calculate overall performance
    total_initial_capital = len(backtest_results) * 1000
    total_final_value = summary_df['portfolio_value'].sum()
    overall_return = ((total_final_value - total_initial_capital) / total_initial_capital) * 100

    print(f"\n\n--- OVERALL PORTFOLIO PERFORMANCE ---")
    print("=" * 50)
    print(f"💰 Total Initial Capital: ${total_initial_capital:,.2f}")
    print(f"📈 Total Final Value: ${total_final_value:,.2f}")
    print(f"🚀 Overall Return: {overall_return:+.2f}%")
    print(f"💸 Total Fees Across All: ${summary_df['total_fees'].sum():,.2f}")


--- GRID TRADING BACKTEST EXECUTION ---
🎯 Testing Grid Strategy pada 5 Cryptocurrencies
💰 Initial Capital: $1,000 per crypto
----------------------------------------------------------------------

===== BACKTESTING BTC =====
Grid Spacing: 1.04%
----------------------------------------
Grid Strategy Initialized:
  Capital: $1,000.00
  Grid Spacing: 1.04%
  Grid Levels: 8 above/below

Grid Setup Complete:
  Base Price: $94,419.76
  Capital per Grid: $100.00
  📈 Highest Sell Level: $102,275.48
  📉 Lowest Buy Level: $86,564.03
   Running backtest...

   ✅ Backtest Complete!
      Total Orders Executed: 56
      Final Portfolio Value: $1,079.28
      Total Return: $79.28
      Return Percentage: +7.93%
      Cash Remaining: $1,079.28
      Crypto Holdings: 0
      Total Fees Paid: $5.68

===== BACKTESTING ETH =====
Grid Spacing: 1.36%
----------------------------------------
Grid Strategy Initialized:
  Capital: $1,000.00
  Grid Spacing: 1.36%
  Grid Levels: 8 above/below

Grid Setup Compl